# Relation of schooling years to education

In this notebook we will analyze the correlation between years of education and the proficiency in certain skills.

First we will load all the data and extract the relevant indicators (see notebook `11` for details):

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
import edstats_utils as edstats
%matplotlib inline

# some pandas options
pd.set_option('display.max_colwidth', -1)

datapath = "./datasets/edstats"
df_country = pd.read_csv(os.path.join(datapath, "EdStatsCountry.csv"))
df_data = pd.read_csv(os.path.join(datapath, "EdStatsData.csv"))
df_series = pd.read_csv(os.path.join(datapath, "EdStatsSeries.csv"))
df_note = pd.read_csv(os.path.join(datapath, "EdStatsFootNote.csv"))
df_cseries = pd.read_csv(os.path.join(datapath, "EdStatsCountry-Series.csv"))

In [2]:
# list of total wittgenstein projection indicators
list_witt_total = edstats.list_indicators('Population in .* highest level .* Total', df_data)
#list_witt_total = ['PRJ.POP.ALL.S1.MF', 'PRJ.POP.ALL.1.MF', 'PRJ.POP.ALL.2.MF', 'PRJ.POP.ALL.3.MF', 'PRJ.POP.ALL.4.MF', 'PRJ.POP.ALL.NED.MF']
list_witt_total

['PRJ.POP.ALL.S1.MF',
 'PRJ.POP.ALL.2.MF',
 'PRJ.POP.ALL.NED.MF',
 'PRJ.POP.ALL.4.MF',
 'PRJ.POP.ALL.1.MF',
 'PRJ.POP.ALL.3.MF']